# Cистемы прогнозирования качества молока и объем удоя коров для покупки их молочному хозяйству «Вольный луг»

содержание

_____
_____
## Описание

Молочное хозяйство «Вольный луг», ориентированное на расширение поголовья коров, заключило контракт с ассоциацией пастбищ «ЭкоФерма». Фермер стремится создать стабильное и высококачественное стадо, отбирая бурёнок, которые не только обеспечат высокий удой, но и удовлетворят строгим требованиям к вкусовым качествам молока. Для этого требуется система, которая поможет автоматизировать процесс отбора, снижая риски и обеспечивая объективность в принятии решения.

- Цели и задачи проекта:

    - Прогнозирование удоя коровы: Разработка модели машинного обучения, которая на основе данных о корове (возраст, порода, физическое состояние, рацион, генетика и т.д.) будет прогнозировать её потенциальный удой за год. Модель должна обеспечивать точность предсказания с учётом различных факторов, влияющих на продуктивность.

    - Оценка качества молока:
    Вторая модель будет прогнозировать вероятность того, что молоко от данной коровы будет соответствовать требованиям фермерского хозяйства по вкусовым качествам. Это может включать параметры, такие как жирность молока, соотношение белков и жиров, а также вкусовые характеристики, оцененные по шкале фермером.

    - Критерии отбора коров:
    Для фермерского хозяйства критически важно, чтобы каждая покупаемая бурёнка:
        - Обеспечивала удой не менее 6000 кг молока в год.
        - Производила молоко, соответствующее строгим критериям вкуса.

_____
_____
## Импорт и подготовка к работе

In [1]:
# %pip install bidict

In [2]:
from bidict import bidict
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
pd.set_option('display.max_columns', None)

_____
_____
## Загрузка данных и общая информация

_____
### Скачивание датасетов, общая информация и первичная обработка

#### ```ferma_main.csv```

|Поле            |Описание                                                           |
|----------------|-------------------------------------------------------------------|
|id              |Уникальный идентификатор коровы                                    |
|Удой, кг        |Масса молока, которую корова даёт в год (в килограммах)            |
|ЭКЕ             |Показатель питательности корма коровы                              |
|Сырой протеин, г|Содержание сырого протеина в корме (в граммах)                     |
|СПО             |Отношение сахара к протеину в корме коровы                         |
|Порода          |Порода коровы                                                      |
|Тип пастбища    |Ландшафт лугов, на которых паслась корова                          |
|порода папы_быка|Порода папы коровы                                                 |
|Жирность, %     |Содержание жиров в молоке (в процентах)                            |
|Белок, %        |Содержание белков в молоке (в процентах)                           |
|Вкус молока     |Оценка вкуса молока по личным критериям фермера (вкусно, не вкусно)|
|Возраст         |Возраст коровы (менее 2 лет, более 2 лет)                          |

In [4]:
ferma = pd.read_csv('ferma_main.csv', delimiter=';', decimal=',')
display(ferma.head(10))
ferma.info()

,id,"Удой, кг",ЭКЕ (Энергетическая кормовая единица),"Сырой протеин, г",СПО (Сахаро-протеиновое соотношение),Порода,Тип пастбища,порода папы_быка,"Жирность,%","Белок,%",Вкус молока,Возраст
0,1,5863,14.2,1743,0.890,Вис Бик Айдиал,Равнинное,Айдиал,3.58,3.076,вкусно,более_2_лет
1,2,5529,12.8,2138,0.890,Вис Бик Айдиал,Равнинные,Соверин,3.54,3.079,вкусно,менее_2_лет
2,3,5810,14.0,1854,0.885,РефлешнСоверинг,Холмистое,Соверин,3.59,3.074,не вкусно,более_2_лет
3,4,5895,12.4,2012,0.885,РефлешнСоверинг,Холмистое,Айдиал,3.40,3.075,не вкусно,более_2_лет
4,5,5302,12.8,1675,0.885,Вис Бик Айдиал,Равнинные,Соверин,3.73,3.073,вкусно,менее_2_лет
5,6,5254,12.7,1806,0.885,РефлешнСоверинг,Холмистое,Соверин,3.13,3.078,вкусно,менее_2_лет
6,7,5910,13.2,1792,0.885,Вис Бик Айдиал,Равнинные,Соверин,3.48,3.077,вкусно,более_2_лет
7,8,6179,14.7,1806,0.930,Вис Бик Айдиал,Равнинные,Айдиал,3.55,3.076,вкусно,более_2_лет
8,9,6783,14.2,1918,0.940,РефлешнСоверинг,Холмистое,Айдиал,3.71,3.077,вкусно,более_2_лет
9,10,5339,13.4,1791,0.895,Вис Бик Айдиал,Холмистое,Соверин,3.13,3.075,вкусно,менее_2_лет


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 634 entries, 0 to 633
Data columns (total 12 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     634 non-null    int64  
 1   Удой, кг                               634 non-null    int64  
 2   ЭКЕ (Энергетическая кормовая единица)  634 non-null    float64
 3   Сырой протеин, г                       634 non-null    int64  
 4   СПО (Сахаро-протеиновое соотношение)   634 non-null    float64
 5   Порода                                 634 non-null    object 
 6   Тип пастбища                           634 non-null    object 
 7   порода папы_быка                       634 non-null    object 
 8   Жирность,%                             634 non-null    float64
 9   Белок,%                                634 non-null    float64
 10  Вкус молока                            634 non-null    object 
 11  Возрас

#### ```ferma_dad.csv```

|Поле    |Описание                       |
|--------|-------------------------------|
|id      |Уникальный идентификатор коровы|
|Имя Папы|Имя папы коровы                |

In [5]:
dads = pd.read_csv('ferma_dad.csv', delimiter=';')
display(dads.head())
dads.info()

,id,Имя Папы
0,1,Буйный
1,2,Соловчик
2,3,Барин
3,4,Буйный
4,5,Барин


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 629 entries, 0 to 628
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        629 non-null    int64 
 1   Имя Папы  629 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.0+ KB


#### ```interested_cows.csv```

|Поле                   |Описание                                                   |
|-----------------------|-----------------------------------------------------------|
|Порода                 |Порода коровы                                              |
|Тип пастбища           |Ландшафт лугов, на которых паслась корова                  |
|порода папы_быка       |Порода папы коровы                                         |
|Имя_папы               |Имя папы коровы                                            |
|Текущая_жирность,%     |Содержание жиров в молоке (в процентах)                    |
|Текущий_уровень_белок,%|Содержание белков в молоке (в процентах)                   |
|Возраст                |Возраст коровы, бинарный признак (менее_2_лет, более_2_лет)|


In [6]:
interested_cows = pd.read_csv('cow_buy.csv', delimiter=';', decimal=',')
display(interested_cows.head())
interested_cows.info()

,Порода,Тип пастбища,порода папы_быка,Имя_папы,"Текущая_жирность,%","Текущий_уровень_белок,%",Возраст
0,Вис Бик Айдиал,холмистое,Айдиал,Геркулес,3.58,3.076,более_2_лет
1,Вис Бик Айдиал,равнинное,Соверин,Буйный,3.54,3.081,менее_2_лет
2,РефлешнСоверинг,равнинное,Соверин,Барин,3.59,3.074,более_2_лет
3,РефлешнСоверинг,холмистое,Айдиал,Буйный,3.40,3.061,более_2_лет
4,РефлешнСоверинг,равнинное,Айдиал,Буйный,3.64,3.074,более_2_лет


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Порода                   20 non-null     object 
 1   Тип пастбища             20 non-null     object 
 2   порода папы_быка         20 non-null     object 
 3   Имя_папы                 20 non-null     object 
 4   Текущая_жирность,%       20 non-null     float64
 5   Текущий_уровень_белок,%  20 non-null     float64
 6   Возраст                  20 non-null     object 
dtypes: float64(2), object(5)
memory usage: 1.2+ KB


#### Первичная обработка

Объединим датасеты **```dads```** и **```ferma```**

In [7]:
ferma = pd.merge(ferma, dads, on='id', how='left')
ferma.head()

,id,"Удой, кг",ЭКЕ (Энергетическая кормовая единица),"Сырой протеин, г",СПО (Сахаро-протеиновое соотношение),Порода,Тип пастбища,порода папы_быка,"Жирность,%","Белок,%",Вкус молока,Возраст,Имя Папы
0,1,5863,14.2,1743,0.890,Вис Бик Айдиал,Равнинное,Айдиал,3.58,3.076,вкусно,более_2_лет,Буйный
1,2,5529,12.8,2138,0.890,Вис Бик Айдиал,Равнинные,Соверин,3.54,3.079,вкусно,менее_2_лет,Соловчик
2,3,5810,14.0,1854,0.885,РефлешнСоверинг,Холмистое,Соверин,3.59,3.074,не вкусно,более_2_лет,Барин
3,4,5895,12.4,2012,0.885,РефлешнСоверинг,Холмистое,Айдиал,3.40,3.075,не вкусно,более_2_лет,Буйный
4,5,5302,12.8,1675,0.885,Вис Бик Айдиал,Равнинные,Соверин,3.73,3.073,вкусно,менее_2_лет,Барин


_____
### Вывод:

- ```ferma``` (```ferma_main.csv```)
    - Содержит данные о стаде фермера. Включает информацию о коровах, их породах, питании, молочной продуктивности и вкусе молока. Структура данных позволяет анализировать взаимосвязь между кормом, породой, возрастом и качеством молока.
    - Всего 634 строки.
    - Нет пропущенных значений.
    - 3 количественных признака: ```Удой, кг```; ```Сырой протеин, г```; ```Жирность, %```, ```Белок, %```.
    - 9 категориальных признаков: ```ЭКЕ```, ```СПО```, ```Порода```, ```Тип пастбища```, ```Порода папы_быка```, ```Вкус молока```, ```Возраст```.
    - Типы данных верные.
    - Надо переименовать имена столбцов.

- ```dads``` (```ferma_dad.csv```) - Объединил с ```ferma``` по ```id```
    - Хранит имя папы каждой коровы. Этот датасет позволяет связать коров с их отцами для дальнейшего анализа генетических характеристик.
    - Всего 629 строк.
    - Нет пропущенных значений.
    - 1 количественный признак: ```id```.
    - 1 категориальный признак: ```Имя Папы```.
    - Типы данных верные.
    - Надо переименовать имена столбцов.

- ```interested_cows``` (```cow_buy.csv```)
    - Содержит данные о коровах, которых фермер планирует купить. Включает информацию о породе, пастбище, кормлении и молочной продуктивности на момент продажи, но без данных о корме и удое.
    - Всего 20 строк.
    - Нет пропущенных значений.
    - 0 количественных признаков.
    - 7 категориальных признаков: ```Порода```, ```Тип пастбища```, ```порода папы_быка```, ```Имя_папы```, ```Текущая_жирность,%```, ```Текущий_уровень_белок,%```, ```Возраст```.
    - Типы данных верные.
    - Надо переименовать имена столбцов.

_____
_____
## Предобработка данных

_____
### Удаление ненужного столбца id

Так как я слил два датасета по ```id```, то этот признак больше не нужен

In [8]:
ferma = ferma.drop('id', axis=1)
ferma.head()

,"Удой, кг",ЭКЕ (Энергетическая кормовая единица),"Сырой протеин, г",СПО (Сахаро-протеиновое соотношение),Порода,Тип пастбища,порода папы_быка,"Жирность,%","Белок,%",Вкус молока,Возраст,Имя Папы
0,5863,14.2,1743,0.890,Вис Бик Айдиал,Равнинное,Айдиал,3.58,3.076,вкусно,более_2_лет,Буйный
1,5529,12.8,2138,0.890,Вис Бик Айдиал,Равнинные,Соверин,3.54,3.079,вкусно,менее_2_лет,Соловчик
2,5810,14.0,1854,0.885,РефлешнСоверинг,Холмистое,Соверин,3.59,3.074,не вкусно,более_2_лет,Барин
3,5895,12.4,2012,0.885,РефлешнСоверинг,Холмистое,Айдиал,3.40,3.075,не вкусно,более_2_лет,Буйный
4,5302,12.8,1675,0.885,Вис Бик Айдиал,Равнинные,Соверин,3.73,3.073,вкусно,менее_2_лет,Барин


_____
### Переименовывание столбцов датафреймов

1. **```ferma```**

In [9]:
ferma_columns_translate = bidict({
    'Удой, кг': 'milk_yield',
    'ЭКЕ (Энергетическая кормовая единица)': 'energy_unit',
    'Сырой протеин, г': 'raw_protein',
    'СПО (Сахаро-протеиновое соотношение)': 'sugar_protein_ratio',
    'Порода': 'breed',
    'Тип пастбища': 'pasture_type',
    'порода папы_быка': 'dad_breed',
    'Жирность,%': 'fat_content',
    'Белок,%': 'protein_content',
    'Вкус молока': 'milk_taste',
    'Возраст': 'age',
    'Имя Папы': 'dad_name'
})

ferma = ferma.rename(columns=ferma_columns_translate)
ferma.head()

,milk_yield,energy_unit,raw_protein,sugar_protein_ratio,breed,pasture_type,dad_breed,fat_content,protein_content,milk_taste,age,dad_name
0,5863,14.2,1743,0.890,Вис Бик Айдиал,Равнинное,Айдиал,3.58,3.076,вкусно,более_2_лет,Буйный
1,5529,12.8,2138,0.890,Вис Бик Айдиал,Равнинные,Соверин,3.54,3.079,вкусно,менее_2_лет,Соловчик
2,5810,14.0,1854,0.885,РефлешнСоверинг,Холмистое,Соверин,3.59,3.074,не вкусно,более_2_лет,Барин
3,5895,12.4,2012,0.885,РефлешнСоверинг,Холмистое,Айдиал,3.40,3.075,не вкусно,более_2_лет,Буйный
4,5302,12.8,1675,0.885,Вис Бик Айдиал,Равнинные,Соверин,3.73,3.073,вкусно,менее_2_лет,Барин


|Поле               |Описание                                                           |
|-----------------  |-------------------------------------------------------------------|
|milk_yield         |Масса молока, которую корова даёт в год (в килограммах)            |
|energy_unit        |Показатель питательности корма коровы                              |
|raw_protein        |Содержание сырого протеина в корме (в граммах)                     |
|sugar_protein_ratio|Отношение сахара к протеину в корме коровы                         |
|breed              |Порода коровы                                                      |
|pasture_type       |Ландшафт лугов, на которых паслась корова                          |
|dad_breed          |Порода папы коровы                                                 |
|fat_content        |Содержание жиров в молоке (в процентах)                            |
|protein_content    |Содержание белков в молоке (в процентах)                           |
|milk_taste         |Оценка вкуса молока по личным критериям фермера (вкусно, не вкусно)|
|age                |Возраст коровы (менее 2 лет, более 2 лет)                          |
|dad_name           |Имя папы коровы                                                    |

2. **```interested_cows```**

In [10]:
interested_cows_columns_translate = bidict({
    'Порода': 'breed',
    'Тип пастбища': 'pasture_type',
    'порода папы_быка': 'dad_breed',
    'Имя_папы': 'dad_name',
    'Текущая_жирность,%': 'current_fat_content',
    'Текущий_уровень_белок,%': 'current_protein_content',
    'Возраст': 'age'
})

interested_cows = interested_cows.rename(columns=interested_cows_columns_translate)
interested_cows.head()

,breed,pasture_type,dad_breed,dad_name,current_fat_content,current_protein_content,age
0,Вис Бик Айдиал,холмистое,Айдиал,Геркулес,3.58,3.076,более_2_лет
1,Вис Бик Айдиал,равнинное,Соверин,Буйный,3.54,3.081,менее_2_лет
2,РефлешнСоверинг,равнинное,Соверин,Барин,3.59,3.074,более_2_лет
3,РефлешнСоверинг,холмистое,Айдиал,Буйный,3.40,3.061,более_2_лет
4,РефлешнСоверинг,равнинное,Айдиал,Буйный,3.64,3.074,более_2_лет


|Поле                   |Описание                                  |
|-----------------------|------------------------------------------|
|breed                  |Порода коровы                             |
|pasture_type           |Ландшафт лугов, на которых паслась корова |
|dad_breed              |Порода папы коровы                        |
|dad_name               |Имя папы коровы                           |
|current_fat_content    |Содержание жиров в молоке (в процентах)   |
|current_protein_content|Содержание белков в молоке (в процентах)  |
|age                    |Возраст коровы (менее 2 лет, более 2 лет) |

_____
### Обработка дубликатов

#### Явные

1. **```ferma```**

In [11]:
print('Кол-во явных дубликатов:', ferma.duplicated().sum())
ferma = ferma.drop_duplicates()

Кол-во явных дубликатов: 5


2. **```interested_cows```**

In [12]:
print('Кол-во явных дубликатов:', interested_cows.duplicated().sum())
interested_cows = interested_cows.drop_duplicates()

Кол-во явных дубликатов: 4


#### Неявные

1. **```ferma```**

In [13]:
for categorical_col in ferma.select_dtypes(include='object').columns:
    print(f'Уникальные значения {categorical_col:<12}\t:\t{ferma[categorical_col].unique()}')

Уникальные значения breed       	:	['Вис Бик Айдиал' 'РефлешнСоверинг']
Уникальные значения pasture_type	:	['Равнинное' 'Равнинные' 'Холмистое']
Уникальные значения dad_breed   	:	['Айдиал' 'Соверин' 'Айдиалл']
Уникальные значения milk_taste  	:	['вкусно' 'не вкусно']
Уникальные значения age         	:	['более_2_лет' 'менее_2_лет']
Уникальные значения dad_name    	:	['Буйный' 'Соловчик' 'Барин' 'Геркулес']


- В столбце ```dad_breed``` есть неявный дубликат ("Айдиал" и "Айдиалл")

In [14]:
ferma.dad_breed = ferma.dad_breed.replace('Айдиалл', 'Айдиал')
print(f'Уникальные значения dad_breed: {ferma.dad_breed.unique()}')

Уникальные значения dad_breed: ['Айдиал' 'Соверин']


2. **```interested_cows```**

In [15]:
for categorical_col in interested_cows.select_dtypes(include='object').columns:
    print(f'Уникальные значения {categorical_col:<12}\t:\t{interested_cows[categorical_col].unique()}')

Уникальные значения breed       	:	['Вис Бик Айдиал' 'РефлешнСоверинг']
Уникальные значения pasture_type	:	['холмистое' 'равнинное']
Уникальные значения dad_breed   	:	['Айдиал' 'Соверин']
Уникальные значения dad_name    	:	['Геркулес' 'Буйный' 'Барин' 'Соловчик']
Уникальные значения age         	:	['более_2_лет' 'менее_2_лет']


Неявные дубликаты отсутствуют

_____
### Обработка аномальных значений

1. **```ferma```**

In [16]:
fig = make_subplots(
    rows=2, cols=3,
    vertical_spacing=0.07,
    subplot_titles=[
        ferma_columns_translate.inverse[name] for name in ferma.select_dtypes(include=['number']).columns
    ])

for i, column in enumerate(ferma.select_dtypes(include=['number']).columns):
    fig.add_trace(
        go.Box(y=ferma[column], name=''),
        row=(i // 3) + 1, col=(i % 3) + 1
    )
fig.update_layout(height=1000, width=1000, showlegend=False, title='График "Ящик с усами" для каждой количественной фичи в ferma')

- "В среднем, корова может дать от 5 000 до 10 000 литров молока в год в зависимости от породы, рациона и условий содержания. Однако, более продуктивные породы могут достигать удоя до 12 000-15 000 литров в год."
- Выбросом можно считать только значение 45616 в столбце ```Удой, кг``` (```milk_yield```). 
- Хоть математически есть выбросы и в других признаках датасета, однако они в рамках реальности считаются вполне нормальными.

In [17]:
display(ferma[ferma.milk_yield > 15000])
ferma = ferma[ferma.milk_yield < 15000]

,milk_yield,energy_unit,raw_protein,sugar_protein_ratio,breed,pasture_type,dad_breed,fat_content,protein_content,milk_taste,age,dad_name
16,45616,11.5,1675,0.93,Вис Бик Айдиал,Холмистое,Айдиал,3.22,3.076,вкусно,более_2_лет,Буйный


2. **```interested_cows```**

In [18]:
fig = make_subplots(
    rows=1, cols=2,
    vertical_spacing=0.07,
    subplot_titles=[
        interested_cows_columns_translate.inverse[name] for name in interested_cows.select_dtypes(include=['number']).columns
    ])

for i, column in enumerate(interested_cows.select_dtypes(include=['number']).columns):
    fig.add_trace(
        go.Box(y=interested_cows[column], name=''),
        row=(i // 3) + 1, col=(i % 3) + 1
    )
fig.update_layout(height=550, width=750, showlegend=False, title='График "Ящик с усами" для каждой количественной фичи в interested_cows')

Выбросов нет.

_____
### Общая обработка данных ```interested_cows```

В ```interested_cows``` параметры корма ```ЭКЕ (Энергетическая кормовая единица)```, ```Сырой протеин, г``` и ```СПО (Сахаро-протеиновое соотношение)``` отсутствуют. Технологи заказчика пересмотрели подход к кормлению: для новых коров планируется увеличить значения каждого из этих параметров на 5%. Поэтому добавим эти три признака, взяв данные из ```ferma```.

In [19]:
interested_cows['energy_unit'] = ferma.energy_unit.mean() * 1.05
interested_cows['raw_protein'] = ferma.raw_protein.mean() * 1.05
interested_cows['sugar_protein_ratio'] = ferma.sugar_protein_ratio.mean() * 1.05
interested_cows.head()

,breed,pasture_type,dad_breed,dad_name,current_fat_content,current_protein_content,age,energy_unit,raw_protein,sugar_protein_ratio
0,Вис Бик Айдиал,холмистое,Айдиал,Геркулес,3.58,3.076,более_2_лет,15.276162,2019.947532,0.958744
1,Вис Бик Айдиал,равнинное,Соверин,Буйный,3.54,3.081,менее_2_лет,15.276162,2019.947532,0.958744
2,РефлешнСоверинг,равнинное,Соверин,Барин,3.59,3.074,более_2_лет,15.276162,2019.947532,0.958744
3,РефлешнСоверинг,холмистое,Айдиал,Буйный,3.40,3.061,более_2_лет,15.276162,2019.947532,0.958744
4,РефлешнСоверинг,равнинное,Айдиал,Буйный,3.64,3.074,более_2_лет,15.276162,2019.947532,0.958744


_____
### Вывод:

- ```ferma```:
    - Кол-во строк в сократилось с 634 до 628.
    - Удален столбец ```id```, из-за дальнейшей ненадобности.
    - Переименованы столбцы на корректные имена.
    - Обработаны дубликаты и аномальные значения.

|Старые имена        |Новые имена          |Описание                                                           |
|--------------------|---------------------|-------------------------------------------------------------------|
|~~id~~              |**-**                |Уникальный идентификатор коровы                                    |
|~~Удой, кг~~        |*milk_yield*         |Масса молока, которую корова даёт в год (в килограммах)            |
|~~ЭКЕ~~             |*energy_unit*        |Показатель питательности корма коровы                              |
|~~Сырой протеин, г~~|*raw_protein*        |Содержание сырого протеина в корме (в граммах)                     |
|~~СПО~~             |*sugar_protein_ratio*|Отношение сахара к протеину в корме коровы                         |
|~~Порода~~          |*breed*              |Порода коровы                                                      |
|~~Тип пастбища~~    |*pasture_type*       |Ландшафт лугов, на которых паслась корова                          |
|~~порода папы_быка~~|*dad_breed*          |Порода папы коровы                                                 |
|~~Жирность, %~~     |*fat_content*        |Содержание жиров в молоке (в процентах)                            |
|~~Белок, %~~        |*protein_content*    |Содержание белков в молоке (в процентах)                           |
|~~Вкус молока~~     |*milk_taste*         |Оценка вкуса молока по личным критериям фермера (вкусно, не вкусно)|
|~~Возраст~~         |*age*                |Возраст коровы (менее 2 лет, более 2 лет)                          |
|**-**               |*dad_name*           |Имя папы коровы                                                    |

- ```interested_cows```:
    - Кол-во строк в сократилось с 20 до 18.
    - Переименованы столбцы на корректные имена.
    - Обработаны дубликаты.
    - Добавлены признаки ```energy_unit```, ```raw_protein```, ```sugar_protein_ratio``` по пересмотренному технологами подходу к кормлению коров

|Старые имена               |Новые имена              |Описание                                  |
|---------------------------|-------------------------|------------------------------------------|
|~~Порода~~                 |*breed*                  |Порода коровы                             |
|~~Тип пастбища~~           |*pasture_type*           |Ландшафт лугов, на которых паслась корова |
|~~порода папы_быка~~       |*dad_breed*              |Порода папы коровы                        |
|~~Имя_папы~~               |*dad_name*               |Имя папы коровы                           |
|~~Текущая_жирность,%~~     |*current_fat_content*    |Содержание жиров в молоке (в процентах)   |
|~~Текущий_уровень_белок,%~~|*current_protein_content*|Содержание белков в молоке (в процентах)  |
|~~Возраст~~                |*age*                    |Возраст коровы (менее 2 лет, более 2 лет) |

_____
_____
## Исследовательский анализ данных

_____
### Функции отрисовок данных

_____
### Вывод:

- 

_____
_____
## Статистический анализ данных

_____
### 

_____
### Вывод:

- 

_____
_____
## Модели

_____
### первая (регрессионная)

_____
### втоорая (классификационная)

_____
_____
## Общий вывод

_____
-